In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
(include-book "verified-agent")
(include-book "llm-client"
              :ttags ((:quicklisp) (:quicklisp.osicat) (:quicklisp.dexador) (:http-json) (:llm-client)))
(include-book "mcp-client"
              :ttags ((:quicklisp) (:quicklisp.dexador) (:http-json) (:mcp-client)))

Note (from clause-processors/equality): disabling DISJOIN, DISJOIN2,
CONJOIN and CONJOIN2.


Summary
Form:  ( INCLUDE-BOOK "verified-agent" ...)
Rules: NIL
Time:  0.93 seconds (prove: 0.00, print: 0.00, other: 0.93)
 "/workspaces/verified-agent/src/verified-agent.lisp"

TTAG NOTE (for included book): Adding ttag :QUICKLISP from book /home/acl2/books/quicklisp/base.

TTAG NOTE (for included book): Adding ttag :QUICKLISP.DEXADOR from book /home/acl2/books/quicklisp/dexador.

TTAG NOTE (for included book): Adding ttag :HTTP-JSON from book /workspaces/verified-agent/src/http-json.

TTAG NOTE (for included book): Adding ttag :LLM-CLIENT from book /workspaces/verified-agent/src/llm-client.

Summary
Form:  ( INCLUDE-BOOK "llm-client" ...)
Rules: NIL
Time:  0.44 seconds (prove: 0.00, print: 0.00, other: 0.44)
 "/workspaces/verified-agent/src/llm-client.lisp"

TTAG NOTE (for included book): Adding ttag :QUICKLISP from book /home/acl2/books/quicklisp/base.

TTAG NOTE (for included book): Adding 

In [3]:
;; Model preferences in order - first match wins
(defconst *model-prefs* '("qwen" "nemotron" "llama" "gemma"))


Summary
Form:  ( DEFCONST *MODEL-PREFS* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *MODEL-PREFS*


In [4]:
;; Default system prompt for the ReAct agent
(defconst *default-system-prompt*
  "You are a helpful AI assistant running inside a formally verified ReAct agent framework built in ACL2.

You have access to ACL2 code execution. To run ACL2 code, put it in a fenced code block:

```acl2
(+ 1 2 3)
```

or:

```lisp
(defun factorial (n) (if (zp n) 1 (* n (factorial (1- n)))))
```

I will execute the code and show you the result. You can then continue reasoning or give a final answer.

Be concise. Show your reasoning.")


Summary
Form:  ( DEFCONST *DEFAULT-SYSTEM-PROMPT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *DEFAULT-SYSTEM-PROMPT*


In [5]:
;; Default agent configuration
(defconst *default-agent-config*
  (make-agent-state 
    :max-steps 20              ; Allow up to 20 conversation turns
    :token-budget 50000        ; Token budget for tool calls
    :time-budget 3600          ; 1 hour time budget
    :file-access 1             ; Read access to files
    :execute-allowed t         ; Code execution enabled
    :max-context-tokens 8000   ; Context window size
    :satisfaction 0))          ; Starting satisfaction


Summary
Form:  ( DEFCONST *DEFAULT-AGENT-CONFIG* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *DEFAULT-AGENT-CONFIG*


In [6]:
(defun show-messages (msgs)
  "Display a list of chat messages."
  (declare (xargs :mode :program))
  (if (endp msgs)
      nil
    (let* ((msg (car msgs))
           (role (chat-message->role msg))
           (content (chat-message->content msg))
           (role-str (chat-role-case role
                       :system "SYSTEM"
                       :user "USER"
                       :assistant "ASSISTANT"
                       :tool "TOOL")))
      (prog2$ (cw "~%[~s0]~%~s1~%" role-str content)
              (show-messages (cdr msgs))))))


Summary
Form:  ( DEFUN SHOW-MESSAGES ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 SHOW-MESSAGES


In [7]:
(defun show-conversation (st)
  "Display the full conversation from an agent state."
  (declare (xargs :mode :program))
  (prog2$ (cw "~%========== Conversation ==========")
          (prog2$ (show-messages (get-messages st))
                  (cw "~%===================================~%"))))


Summary
Form:  ( DEFUN SHOW-CONVERSATION ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 SHOW-CONVERSATION


In [8]:
(defun show-context-usage (st)
  "Display context token usage statistics."
  (declare (xargs :mode :program))
  (let* ((msgs (get-messages st))
         (char-len (messages-char-length msgs))
         (token-est (messages-token-estimate msgs))
         (max-tokens (agent-state->max-context-tokens st)))
    (prog2$ (cw "~%Context Usage:~%")
      (prog2$ (cw "  Total characters: ~x0~%" char-len)
        (prog2$ (cw "  Estimated tokens: ~x0~%" token-est)
          (prog2$ (cw "  Max tokens: ~x0~%" max-tokens)
            (cw "  Fits in context: ~x0~%" 
                (messages-fit-p msgs max-tokens))))))))


Summary
Form:  ( DEFUN SHOW-CONTEXT-USAGE ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 SHOW-CONTEXT-USAGE


In [9]:
;; Strip leading whitespace from character list
(defun strip-leading-ws (lst)
  (declare (xargs :mode :program))
  (if (endp lst) nil
    (if (member (car lst) '(#\Space #\Tab #\Newline #\Return))
        (strip-leading-ws (cdr lst))
      lst)))


Summary
Form:  ( DEFUN STRIP-LEADING-WS ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 STRIP-LEADING-WS


In [10]:
;; Trim whitespace from string
(defun my-string-trim (str)
  (declare (xargs :mode :program))
  (let* ((chars (coerce str 'list))
         (trimmed (strip-leading-ws chars))
         (rev-trimmed (strip-leading-ws (reverse trimmed))))
    (coerce (reverse rev-trimmed) 'string)))


Summary
Form:  ( DEFUN MY-STRING-TRIM ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 MY-STRING-TRIM


In [11]:
;; Extract first ```acl2 or ```lisp code block from text
(defun extract-code-block (response)
  "Extract the first ACL2/Lisp code block from a response string."
  (declare (xargs :mode :program))
  (let* ((acl2-start (search "```acl2" response))
         (lisp-start (search "```lisp" response))
         (start-marker (cond ((and acl2-start lisp-start)
                              (if (< acl2-start lisp-start) "```acl2" "```lisp"))
                             (acl2-start "```acl2")
                             (lisp-start "```lisp")
                             (t nil)))
         (start-pos (cond ((and acl2-start lisp-start)
                           (min acl2-start lisp-start))
                          (acl2-start acl2-start)
                          (lisp-start lisp-start)
                          (t nil))))
    (if (not start-pos)
        (mv nil "")
      (let* ((content-start (+ start-pos (length start-marker)))
             (newline-pos (search (coerce '(#\Newline) 'string)
                                  (subseq response content-start (length response))))
             (code-start (if newline-pos (+ content-start newline-pos 1) content-start))
             (rest (subseq response code-start (length response)))
             (end-pos (search "```" rest)))
        (if (not end-pos)
            (mv nil "")
          (mv t (my-string-trim (subseq rest 0 end-pos))))))))


Summary
Form:  ( DEFUN EXTRACT-CODE-BLOCK ...)
Rules: NIL
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
 EXTRACT-CODE-BLOCK


In [12]:
;; Execute ACL2 code via MCP
(defun execute-acl2-code (code mcp-conn state)
  "Execute ACL2 code via MCP connection."
  (declare (xargs :mode :program :stobjs state))
  (if (not (mcp-connection-p mcp-conn))
      (mv "No MCP connection" "" state)
    (mcp-acl2-execute mcp-conn code state)))


Summary
Form:  ( DEFUN EXECUTE-ACL2-CODE ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 EXECUTE-ACL2-CODE


In [13]:
(defun react-loop (agent-st model-id mcp-conn max-steps state)
  "Execute ReAct loop: LLM -> extract code -> execute -> repeat until no code block"
  (declare (xargs :mode :program :stobjs state))
  (if (zp max-steps)
      (prog2$ (cw "~%[Max steps reached]~%")
              (mv agent-st state))
    (mv-let (err response state)
      (llm-chat-completion model-id (get-messages agent-st) state)
      (if err
          (prog2$ (cw "~%LLM Error: ~s0~%" err)
                  (mv agent-st state))
        (let ((agent-st (add-assistant-msg response agent-st)))
          (prog2$ (cw "~%Assistant: ~s0~%" response)
            (mv-let (found? code)
              (extract-code-block response)
              (if (not found?)
                  ;; No code block - done with this turn
                  (mv agent-st state)
                ;; Execute code and continue
                (prog2$ (cw "~%[Executing: ~s0]~%" code)
                  (mv-let (exec-err result state)
                    (execute-acl2-code code mcp-conn state)
                    (let* ((tool-result (if exec-err
                                            (concatenate 'string "Error: " exec-err)
                                          result))
                           ;; Truncate long results for display
                           (display-result (if (> (length tool-result) 300)
                                               (concatenate 'string 
                                                 (subseq tool-result 0 300) "...")
                                             tool-result))
                           (agent-st (add-tool-result tool-result agent-st)))
                      (prog2$ (cw "~%Result: ~s0~%" display-result)
                        ;; Continue ReAct loop
                        (react-loop agent-st model-id mcp-conn (1- max-steps) state)))))))))))))


Summary
Form:  ( DEFUN REACT-LOOP ...)
Rules: NIL
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
 REACT-LOOP


In [14]:
(defun chat-turn (user-msg agent-st model-id mcp-conn state)
  "Execute one chat turn with ReAct: add user message, run ReAct loop"
  (declare (xargs :mode :program :stobjs state))
  (let ((st-with-user (add-user-msg user-msg agent-st)))
    (react-loop st-with-user model-id mcp-conn 5 state)))


Summary
Form:  ( DEFUN CHAT-TURN ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 CHAT-TURN


In [15]:
(defun read-line-chars (acc state)
  "Accumulate characters until newline or EOF."
  (declare (xargs :mode :program :stobjs state))
  (mv-let (char state)
    (read-char$ *standard-ci* state)
    (cond ((null char)  ; EOF
           (mv (if (null acc) nil (coerce (reverse acc) 'string)) state))
          ((eql char #\Newline)
           (mv (coerce (reverse acc) 'string) state))
          (t (read-line-chars (cons char acc) state)))))


Summary
Form:  ( DEFUN READ-LINE-CHARS ...)
Rules: NIL
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
 READ-LINE-CHARS


In [16]:
(defun read-line-from-user (state)
  "Read a line from standard input, return (mv line state)."
  (declare (xargs :mode :program :stobjs state))
  (read-line-chars nil state))


Summary
Form:  ( DEFUN READ-LINE-FROM-USER ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 READ-LINE-FROM-USER


In [17]:
(defun interactive-chat-loop-aux (agent-st model-id mcp-conn state)
  "Helper for interactive chat loop with code execution."
  (declare (xargs :mode :program :stobjs state))
  (prog2$ (cw "~%You: ")
    (mv-let (input state)
      (read-line-from-user state)
      (if (or (null input)
              (equal input "/exit")
              (equal input "/quit"))
          (prog2$ (cw "~%Goodbye!~%")
                  (mv agent-st state))
        (mv-let (new-agent state)
          (chat-turn input agent-st model-id mcp-conn state)
          (interactive-chat-loop-aux new-agent model-id mcp-conn state))))))


Summary
Form:  ( DEFUN INTERACTIVE-CHAT-LOOP-AUX ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 INTERACTIVE-CHAT-LOOP-AUX


In [18]:
(defun interactive-chat-loop (agent-st model-id state)
  "Run an interactive ReAct chat loop with code execution. Type /exit to quit."
  (declare (xargs :mode :program :stobjs state))
  (prog2$ (cw "~%========================================~%")
    (prog2$ (cw "Interactive ReAct Chat with Code Execution~%")
      (prog2$ (cw "(type /exit to quit)~%")
        (prog2$ (cw "========================================~%")
          (prog2$ (cw "~%Connecting to MCP server...~%")
            (mv-let (mcp-err mcp-conn state)
              (mcp-connect *mcp-default-endpoint* state)
              (if mcp-err
                  (prog2$ (cw "MCP connection failed: ~s0~%" mcp-err)
                    (prog2$ (cw "Code execution disabled.~%")
                      (interactive-chat-loop-aux agent-st model-id nil state)))
                (prog2$ (cw "MCP connected.~%")
                  (prog2$ (if (mcp-connection-has-acl2-session-p mcp-conn)
                              (cw "ACL2 session: ~s0~%" 
                                  (mcp-connection->acl2-session-id mcp-conn))
                            (cw "Warning: No persistent ACL2 session (slower mode).~%"))
                    (interactive-chat-loop-aux agent-st model-id mcp-conn state)))))))))))


Summary
Form:  ( DEFUN INTERACTIVE-CHAT-LOOP ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 INTERACTIVE-CHAT-LOOP


In [19]:
;; Initialize fresh agent state
(defconst *initial-chat-state*
  (init-agent-conversation *default-system-prompt* *default-agent-config*))


Summary
Form:  ( DEFCONST *INITIAL-CHAT-STATE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *INITIAL-CHAT-STATE*


In [20]:
#-skip-interactive
(make-event
 (mv-let (err models state)
   (llm-list-models-full state)
   (declare (ignorable err))
   (let* ((selected (select-completions-model models *model-prefs*))
          (model-id (if selected (model-info->id selected) "no-model-found")))
     (prog2$
      (if selected
          (cw "~%Selected model: ~s0 (context: ~x1 tokens)~%"
              model-id
              (model-info->loaded-context-length selected))
        (cw "~%WARNING: No model found. Ensure LM Studio is running.~%"))
      (mv nil `(defconst *chat-model* ,model-id) state)))))


Selected model: qwen/qwen3-coder-30b (context: 80000 tokens)

Summary
Form:  ( DEFCONST *CHAT-MODEL* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
 *CHAT-MODEL*


In [21]:
#-skip-interactive
(make-event
 (mv-let (mcp-err mcp-conn state)
   (mcp-connect *mcp-default-endpoint* state)
   (prog2$
    (if mcp-err
        (cw "~%MCP connection failed: ~s0~%Code execution disabled.~%" mcp-err)
      (prog2$
       (cw "~%MCP connected.~%")
       (if (mcp-connection-has-acl2-session-p mcp-conn)
           (cw "ACL2 session: ~s0~%" (mcp-connection->acl2-session-id mcp-conn))
         (cw "No persistent ACL2 session.~%"))))
    (mv nil `(defconst *chat-mcp* ',mcp-conn) state))))


MCP connected.
ACL2 session: 4239f992-d738-4385-a1ec-7f77c557ba74

Summary
Form:  ( DEFCONST *CHAT-MCP* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CHAT-MCP*


In [22]:
#-skip-interactive
(defconst *chat-agent* *initial-chat-state*)


Summary
Form:  ( DEFCONST *CHAT-AGENT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CHAT-AGENT*


In [23]:
#-skip-interactive
(defun chat-fn (msg state)
  "Send a message and get a response. Updates *chat-agent* state."
  (declare (xargs :mode :program :stobjs state))
  (prog2$ (cw "~%You: ~s0~%" msg)
    (mv-let (new-agent state)
      (chat-turn msg *chat-agent* *chat-model* *chat-mcp* state)
      (mv nil
          `(progn (redef+)
                  (defconst *chat-agent* ',new-agent)
                  (redef-))
          state))))


Summary
Form:  ( DEFUN CHAT-FN ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 CHAT-FN


In [24]:
#-skip-interactive
(defmacro chat (msg)
  "Send a message to the chat agent. Example: (chat \"Hello!\")"
  `(make-event (chat-fn ,msg state)))


Summary
Form:  ( DEFMACRO CHAT ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 CHAT


In [25]:
#-skip-interactive
(defmacro chat-reset ()
  "Reset the conversation to initial state."
  '(progn
     (redef+)
     (defconst *chat-agent* *initial-chat-state*)
     (redef-)
     (value-triple :chat-reset)))


Summary
Form:  ( DEFMACRO CHAT-RESET ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 CHAT-RESET
